In [1]:
import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV



In [2]:
t = pd.read_csv("./reports/data.csv")
t = t.dropna()

In [3]:
__type = "math"#sys.argv[1]
t[__type] =  t["labels"].str.contains(__type)
t[__type] = pd.to_numeric(t[__type], downcast="signed")

In [4]:
feature = "problem-statement"

l =t['ps']= t['problem-statement'].tolist()
l2 = t['os']= t['output-spec'].tolist()
res1 = []
res2 = []
regex = re.compile("^a-zA-Z0-9\ '" )
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        res1.append(clean_s(l[i]))
    except:
        res1.append("")
for i in range(len(l2)):
    try:
        res2.append(clean_s(l2[i]))
    except:
        res2.append("")

In [5]:

output = t[[__type,'ps','os' ]]
output = output.dropna()

output[__type] = output[__type].astype(int)

a = output.loc[output[__type] == 1.0]
b = output.loc[output[__type] == 0.0]
b = b.sample(n=len(a), random_state = 42)

In [6]:
output = a.append(b)

In [7]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.optimizers import Adam

max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 500
filters = 250
kernel_size = 3
hidden_dims = 2500
epochs = 2


temp1 =  [hashing_trick(a, max_features) for a in output['ps'].tolist()]
temp2 =  [hashing_trick(a, max_features) for a in output['os'].tolist()]
pad1= sequence.pad_sequences(temp1);
pad2= sequence.pad_sequences(temp2);
temp = np.concatenate((sequence.pad_sequences(temp1), sequence.pad_sequences(temp2)), axis = 1) #np.concatenate((a, b.T), axis=1)
x_train, x_test, y_train, y_test = train_test_split(temp,output[__type], test_size=0.2,random_state=42)



/home/nikepupu/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
class_weight = {0 : len(b),
    1: len(a)}
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test), class_weight=class_weight)

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Accuracy:', acc)
y_pred = model.predict(x_test)
y_pred = [round(value) for value in y_pred]
print(classification_report(y_test, y_pred))

Train...
Train on 1241 samples, validate on 311 samples
Epoch 1/10
1241/1241 [==============================] - 65s 52ms/step - loss: 431.3716 - acc: 0.5028 - val_loss: 0.7054 - val_acc: 0.4984
Epoch 2/10
1241/1241 [==============================] - 63s 51ms/step - loss: 412.3326 - acc: 0.5326 - val_loss: 0.6975 - val_acc: 0.5241
Epoch 3/10
1241/1241 [==============================] - 62s 50ms/step - loss: 378.9927 - acc: 0.6164 - val_loss: 0.6646 - val_acc: 0.6077
Epoch 4/10
1241/1241 [==============================] - 62s 50ms/step - loss: 339.2605 - acc: 0.7059 - val_loss: 0.6597 - val_acc: 0.6238
Epoch 5/10
1241/1241 [==============================] - 63s 51ms/step - loss: 285.1191 - acc: 0.7849 - val_loss: 0.7087 - val_acc: 0.5981
Epoch 6/10
1241/1241 [==============================] - 64s 52ms/step - loss: 245.2765 - acc: 0.8171 - val_loss: 0.7069 - val_acc: 0.6077
Epoch 7/10
1241/1241 [==============================] - 63s 51ms/step - loss: 195.1251 - acc: 0.8662 - val_loss: 0.8